# Ders Adı: Python ile Veri Bilimine Giriş
# Konu: YouTube Trend Video İstatistik Verilerinin Analizi
### Ders Sorumlusu: Dr. Öğretim Üyesi Enver Küçükkülahlı
### Dönem: 2022-23 Bahar Dönemi
### Hazırlayan: Büşra Özdemir - 201001082 - Normal Öğretim

Dataset olarak YouTube Trend Olan Videoların İstatistikleri kullanılacaktır. 

Kullanacağımız dataset YouTube üzerinde en trend videoları belirlemek için kullanıcıların video görüntüleme sayısı, beğenme 
sayısı, beğenmeme sayısı, toplam yapılan yorum sayısı, videoda kullanılan taglar gibi kolonları içermektedir. 
Bu gibi alanların kombinasyonu sayesinde yılın en trend videolarını elde edebilir ve gözlemleyebiliriz.

kaggle.com üzerinde farklı ülkelerde olan YouTube Videolarının İstatistik bilgilerinin saklandığı datasetler de mevcuttur.
NOT: Biz bu projemizde İngiltere için olan dataseti(GBvideos.csv) kullanacağız. Dataset indirme linki aşağıdadır.

https://www.kaggle.com/datasets/datasnaek/youtube-new?select=GBvideos.csv  

● NumPy => Python programlama dili için büyük, çok boyutlu dizileri ve matrisleri destekleyen, bu diziler üzerinde çalışacak üst düzey matematiksel işlevler ekleyen bir kitaplıktır.

● Pandas => Bu kütüphane temel olarak zaman etiketli serileri ve sayısal tabloları işlemek için bir veri yapısı oluşturur ve bu şekilde çeşitli işlemler bu veri yapısı üzerinde gerçekleştirilebilir olur.

### 1. Veri Seti Kolonları

In [1]:
import numpy as np
import pandas as pd

df=pd.read_csv("dataset/youtubeGB.csv") 
youtube=df.copy()

In [4]:
youtube.columns # veri setinde bulunan kolonları/alanları gösterir

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

##### --- Veri seti kolonlarının özellikleri ---
##### video_id: videonun id'si
##### trending_date: videonun trend olduğu tarih
##### title: videonun başlığı/adı
##### channel_title: videonun yayınlandığı kanal adı
##### category_id: videonun hangi kategoriye ait olduğunu belirten id
##### publish_time: videonun yayınlanma zamanı (Örnek: 2017-11-10T07:38:29.000Z)
##### tags: videonun içerdiği etiketler
##### views: videonun görüntülenme/izlenme sayısı
##### likes: videonun beğenme sayısı
##### dislikes: videonun beğenmeme sayısı
##### comment_count: videoya atılan yorum sayısı
##### thumbnail_link: videoya ait küçük resim simgesinin linki
##### comments_disabled: videoya ait yorum yapabilme devre dışı bırakıldı mı?
##### ratings_disabled: videonun değerlendirilmesi devre dışı bırakıldı mı?
##### video_error_or_removed: videoda hata var mı veya video kaldırıldı mı?
##### description: videoya ait açıklama

### 2. Temel Listeleme İşlemleri

In [4]:
# İlk 10 kaydın başlığını getirir.
result=youtube["title"].head(10)
print(result)

0        John Lewis Christmas Ad 2017 - #MozTheMonster
1            Taylor Swift: …Ready for It? (Live) - SNL
2           Eminem - Walk On Water (Audio) ft. Beyoncé
3    Goals from Salford City vs Class of 92 and Fri...
4    Dashcam captures truck's near miss with child ...
5                         How My Relationship Started!
6          Ed Sheeran - Perfect (Official Music Video)
7                        CHRISTMAS HAS GONE TO MY HEAD
8     Elbow - Golden Slumbers (John Lewis Advert 2017)
9    Jennifer Lopez - Amor, Amor, Amor (Official Vi...
Name: title, dtype: object

In [3]:
# Son 5 kaydın title, likes, views kolonlarını getirir.
youtube[["title","likes","views"]].tail(5)

,title,likes,views
38911,Enrique Iglesias - MOVE TO MIAMI (Official Vid...,268088,25066952
38912,Jacob Sartorius - Up With It (Official Music V...,61998,1492219
38913,Anne-Marie - 2002 [Official Video],394830,29641412
38914,Eleni Foureira - Fuego - Cyprus - LIVE - First...,151870,14317515
38915,KYLE - Ikuyo feat. 2 Chainz & Sophia Black [A...,18271,607552


In [2]:
# İlk 20 videonun başlığını, beğenme ve beğenmeme sayısının kolonlarını getirir.
youtube.head(20)[["title","likes","dislikes"]]

,title,likes,dislikes
0,John Lewis Christmas Ad 2017 - #MozTheMonster,55681,10247
1,Taylor Swift: …Ready for It? (Live) - SNL,25561,2294
2,Eminem - Walk On Water (Audio) ft. Beyoncé,787420,43420
3,Goals from Salford City vs Class of 92 and Fri...,193,12
4,Dashcam captures truck's near miss with child ...,30,2
5,How My Relationship Started!,52708,1431
6,Ed Sheeran - Perfect (Official Music Video),1634124,21082
7,CHRISTMAS HAS GONE TO MY HEAD,57309,749
8,Elbow - Golden Slumbers (John Lewis Advert 2017),2163,147
9,"Jennifer Lopez - Amor, Amor, Amor (Official Vi...",190084,15015


### 3. Aggregation (Toplulaştırma)

##### Basit toplulaştırma fonksiyonları:
    ● count() => Sorgu tarafından döndürülen kayıtların sayısını hesaplar.
    ● first() => İlk elemanı getirir.
    ● last() => Son elemanı getirir.
    ● mean() => Tüm elemanların ortalamalarını hesaplar.
    ● median() => Tüm elemanların median’ını alır.
    ● min() => Belirtilen elemanlar içerisinden en küçük değeri döndürür.
    ● max() => Belirtilen elemanlar içerisinden en büyük değeri döndürür.
    ● std() => Tüm elemanların standart sapmasını hesaplar.
    ● var() => Tüm elemanların varyansını hesaplar.
    ● sum() => Tüm elemanların toplamlarını alır.

In [4]:
# Beğenme (like) ve beğenmeme (dislike) sayılarının ortalaması bulunur.
result=youtube["likes"].mean()
print("Beğeni sayılarının ortalaması: ",result)

result=youtube["dislikes"].mean()
print("Beğenmeme sayılarının ortalaması: ",result)

Beğeni sayılarının ortalaması:  134519.55349984582
Beğenmeme sayılarının ortalaması:  7612.559975331483


In [9]:
# category_id alanına göre gruplama yapıldıktan sonra numpy ile likes kolonunun ortalaması, max ve min değerleri gelir.
import numpy as np

youtube.groupby("category_id")["likes"].agg([np.mean,np.max,np.min])

,mean,amax,amin
category_id,,,
1,57205.523089,461135,0
2,24608.506944,114841,61
10,272138.508943,5613827,0
15,28629.913858,187524,147
17,39270.313057,1291139,0
19,9674.447917,23791,54
20,39220.127517,1031695,0
22,46273.495899,625174,0
23,78431.168490,1253154,22


In [10]:
# trending_date alanına göre gruplama yapıldıktan sonra numpy ile views kolonunun ortalaması, max ve min değerleri gelir.
import numpy as np

youtube.groupby("trending_date")["views"].agg([np.mean,np.max,np.min]).head(10) # head(10) ile sadece ilk 10 kayıt gelir.

,mean,amax,amin
trending_date,,,
17.01.12,2.585499e+06,143408235,851
17.02.12,1.841747e+06,66637636,890
17.03.12,2.017085e+06,74789251,922
17.04.12,2.157436e+06,80360459,937
17.05.12,2.265379e+06,84281319,972
17.06.12,2.187256e+06,87449453,983
17.07.12,1.962871e+06,89930713,1014
17.08.12,1.908755e+06,91933007,1192
17.09.12,2.197554e+06,93799976,1208


### 3.1. loc ve iloc
#### ● iloc => Satır ya da sütunun index numarasıyla çalışmak istiyorsak kullanılır.
#### ● loc => Satır ya da sütunun ismini belirterek çalışmak istiyorsak kullanılır.

In [3]:
# En çok görüntülenen videonun başlığını gösterir.
result=youtube[(youtube["views"].max()) == youtube["views"]]["title"].iloc[0]
print(result)

Nicky Jam x J. Balvin - X (EQUIS) | Video Oficial | Prod. Afro Bros & Jeon


In [4]:
# Görüntülenme sayısı en az olan videonun başlığını gösterir.
result=youtube[youtube["views"].min() == youtube["views"]]["title"].iloc[0]
print(result)

Mountain Bikers Worried About Military Land Being Fenced Off


### 4. Grouping (Gruplama)

In [11]:
# Kategoriye göre beğeni ortalamalarını sıralı şekilde gösterir.
result=youtube.groupby("category_id").mean().sort_values("likes")["likes"]
print(result)

category_id
19      9674.447917
2      24608.506944
25     25021.685714
26     26639.183091
15     28629.913858
27     38440.687090
20     39220.127517
17     39270.313057
43     41836.900000
22     46273.495899
1      57205.523089
28     61531.127413
23     78431.168490
24     81572.362012
29    271695.733333
10    272138.508943
Name: likes, dtype: float64


C:\Users\pc\AppData\Local\Temp\ipykernel_5520\764360808.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result=youtube.groupby("category_id").mean().sort_values("likes")["likes"]


In [12]:
# Kategoriye göre yorum sayılarını azalan bir şekilde sıralar.
result=youtube.groupby("category_id").sum().sort_values("comment_count",ascending=False)["comment_count"]
print(result)

category_id
10    293013140
24    116897440
22     18713149
23     16718816
1      15590011
17     10056226
25      9386604
29      7672790
26      7356296
20      6166994
28      4029065
27      1636903
15      1404698
2        505085
19       118177
43        80957
Name: comment_count, dtype: int64


C:\Users\pc\AppData\Local\Temp\ipykernel_5520\3705028098.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result=youtube.groupby("category_id").sum().sort_values("comment_count",ascending=False)["comment_count"]


### 5. Apply İşlemleri

In [13]:
# Her video için kullanılan tag sayısını tag_count adında yeni bir kolon oluşturarak gösterir.
def tagCount(tag):
    return len(tag.split("|"))

youtube["tag_count"]=youtube["tags"].apply(tagCount)

youtube[["title","tag_count"]] # Tüm kolonlar yerine sadece title ve oluşturulan tag_count kolonunun gelmesini sağladık.

,title,tag_count
0,John Lewis Christmas Ad 2017 - #MozTheMonster,9
1,Taylor Swift: …Ready for It? (Live) - SNL,36
2,Eminem - Walk On Water (Audio) ft. Beyoncé,6
3,Goals from Salford City vs Class of 92 and Fri...,9
4,Dashcam captures truck's near miss with child ...,1
...,...,...
38911,Enrique Iglesias - MOVE TO MIAMI (Official Vid...,4
38912,Jacob Sartorius - Up With It (Official Music V...,41
38913,Anne-Marie - 2002 [Official Video],17
38914,Eleni Foureira - Fuego - Cyprus - LIVE - First...,8


In [14]:
# En popüler videolar like/dislike oranına göre hesaplanarak listelenir.
def likeDislikeOranHesapla(dataset):
    likesList=list(youtube["likes"]) # videoların likes listesi
    dislikesList=list(youtube["dislikes"]) # videoların dislikes listesi

    liste=list(zip(likesList,dislikesList)) # zip ile tuple listesi şeklinde gelmesini sağladık. 
                                            # Bir video için like ve dislike => (60003, 2337)

    oranListesi=[]

    for like,dislike in liste:
        if(like+dislike)==0: # eğer like ve dislike toplamı 0'sa popülerlik oranını 0 olarak listeye ekliyoruz.
            oranListesi.append(0)
        else: # like ve dislike toplamı 0 değilse popülerlik oranını hesaplayarak(like/(like+dislike)) listeye ekliyoruz. 
            oranListesi.append(like/(like+dislike))

    return oranListesi

youtube["beğeni_orani"]=likeDislikeOranHesapla(youtube)

# beğeni_orani kolonuna göre azalan biçimde sıralanır.
youtube.sort_values("beğeni_orani",ascending=False)[["title","likes","dislikes","beğeni_orani"]]

,title,likes,dislikes,beğeni_orani
36631,Kris Wu – Like That (Official Music Video),57544,0,1.0
7088,HIGHLIGHTS: MK Dons U18s 1-0 Cardiff City U18s,30,0,1.0
5361,QUIZ : Name the Trail : A Christmas Cracker,2,0,1.0
12082,Lima Sopoaga Tribute🔥🔥||The Unsung Hero||💯 2017,50,0,1.0
5338,Bohemian Rhapsody (2018) - Scene Bohemian Rhap...,71,0,1.0
...,...,...,...,...
21547,To Our Daughter,0,0,0.0
22661,【公式】「劇場版ポケットモンスター みんなの物語」予告編1,0,0,0.0
24920,映画『BLEACH』特報【HD】2018年7月20日（金）公開,0,0,0.0
14864,Altwork 1 Minute Video,0,0,0.0


### 6. Şartlı ve Filtreleme Yaparak Listeleme 

In [18]:
# Beğeni sayısı 450.000 ve üzerinde olanların title ve likes kolonlarını getirir.
result=youtube[(youtube["likes"]>=450000)][["title","likes"]]
print(result)
print(f"Toplam kayıt sayısı: {result.count()}") # Beğeni sayısı 450.000 ve üzerinde olan toplam kaç kayıt olduğu bilgisi

                                                   title    likes
2             Eminem - Walk On Water (Audio) ft. Beyoncé   787420
6            Ed Sheeran - Perfect (Official Music Video)  1634124
53                                   Harry Styles - Kiwi   810895
147    Taylor Swift - Call It What You Want (Lyric Vi...   694768
219           Eminem - Walk On Water (Audio) ft. Beyoncé   840642
...                                                  ...      ...
38831  Clean Bandit - Solo feat. Demi Lovato [Officia...   454441
38832               Gorillaz - Humility (Official Video)   911150
38836  BTS (방탄소년단) 'FAKE LOVE' Official MV (Extended ...  1817699
38837  Cardi B, Bad Bunny & J Balvin - I Like It [Off...  1816753
38854     Jennifer Lopez - Dinero ft. DJ Khaled, Cardi B   455949

[2563 rows x 2 columns]
Toplam kayıt sayısı: title    2563
likes    2563
dtype: int64


In [19]:
# Trend tarihi 18.29.12 ve daha büyük olan kayıtların title, trending_date kolonlarını listeler.
youtube[(youtube["trending_date"]>="18.29.12")][["title","trending_date"]]

,title,trending_date
14991,WHEN WE FIRST MET Official Trailer (2018) Alex...,18.30.01
14992,"Ronda Rousey confronts Asuka, Alexa Bliss and ...",18.30.01
14993,Migos - Stir Fry (Official),18.30.01
14994,WEEI: Alex Reimer calls Tom Brady’s daughter a...,18.30.01
14995,HORMONES | The Weekly #4 | LIFESTYLE,18.30.01
...,...,...
36561,Ozuna - Única (Video Oficial) 🐻 A U R A,18.31.05
36562,Ella Mai - Boo'd Up,18.31.05
36563,Miguel - Come Through and Chill (Official Vide...,18.31.05
36564,Alton Brown Rigorously Reviews Spicy Wings | H...,18.31.05


In [8]:
# Beğenme ve beğenmeme sayısı 0 olan kayıtları listeler
result=youtube[(youtube["likes"]==0) & (youtube["dislikes"]==0)][["title","likes","dislikes"]]
print(result)
print(youtube[(youtube["likes"]==0) & (youtube["dislikes"]==0)]["title"].count()) # Bu kayıtların toplam sayısını gösterir.

                                               title  likes  dislikes
75     Every Kevin Spacey Joke from Difficult People      0         0
296    Every Kevin Spacey Joke from Difficult People      0         0
509    Every Kevin Spacey Joke from Difficult People      0         0
749    Every Kevin Spacey Joke from Difficult People      0         0
964    Every Kevin Spacey Joke from Difficult People      0         0
...                                              ...    ...       ...
38214                  This Is America: Women's Edit      0         0
38382                  This Is America: Women's Edit      0         0
38555                  This Is America: Women's Edit      0         0
38730                  This Is America: Women's Edit      0         0
38900                  This Is America: Women's Edit      0         0

[272 rows x 3 columns]
272


In [2]:
# Video görüntülenme sayısı 4.000.000 ile 4.200.000 arasında olan kayıtların title ve views bilgilerini listeler.
youtube[(youtube["views"]>4000000) & (youtube["views"]<=4200000)][["title","views"]]

,title,views
919,FIRST TIME IM DOING THIS! TALKS WITH LIZA.,4039889
1127,FIRST TIME IM DOING THIS! TALKS WITH LIZA.,4101688
1332,FIRST TIME IM DOING THIS! TALKS WITH LIZA.,4149350
1396,블락비 (Block B) - Shall We Dance MV,4000448
1549,FIRST TIME IM DOING THIS! TALKS WITH LIZA.,4191001
...,...,...
38554,Ar'mon and Trey - Forever (Official Music Video),4027975
38616,Chris Brown - Hope You Do (Official Video),4186579
38721,Years & Years - If You're Over Me (Official vi...,4105414
38729,Ar'mon and Trey - Forever (Official Music Video),4145972


In [3]:
# Videoya yapılan yorum sayısı 100 ile 110 arasında olan kayıtların title ve yorum sayısı bilgisini gösterir.
youtube[(youtube["comment_count"]>100) & (youtube["comment_count"]<110)][["title","comment_count"]]

,title,comment_count
437,"Mindy Kaling, Apu & Indian Americans by Hari K...",103
682,"Mindy Kaling, Apu & Indian Americans by Hari K...",109
753,How Are Waves Formed? - Earth Lab,109
1358,Corey Haim's Mom Talks to Dr. Oz,102
1573,Corey Haim's Mom Talks to Dr. Oz,103
...,...,...
28036,Khabib Nurmagomedov Comments on Confrontation ...,105
28727,Limmy's Homemade Show: Radiator Harp,109
29712,Ice Cube Had to Bring His Ex-Girlfriend to Prom,105
29765,West Coast ‘Wives Pack The LA Clubhouse | RHOB...,107


In [4]:
# Kanal adı Star Wars olan tüm kayıtları başlık bilgisiyle beraber listeler.
result=youtube[(youtube["channel_title"]=="Star Wars")][["title","channel_title"]]
print(result)
print("Toplam kayıt sayısı: ",result.count()["channel_title"]) 

                                                   title channel_title
168                 Star Wars: The Last Jedi Awake (:45)     Star Wars
371                 Star Wars: The Last Jedi Awake (:45)     Star Wars
580                 Star Wars: The Last Jedi Awake (:45)     Star Wars
1599                Star Wars: The Last Jedi Tempt (:30)     Star Wars
1807                Star Wars: The Last Jedi Tempt (:30)     Star Wars
...                                                  ...           ...
37056  Arrested Development: Star Wars with Ron Howar...     Star Wars
37220  Arrested Development: Star Wars with Ron Howar...     Star Wars
37388  Arrested Development: Star Wars with Ron Howar...     Star Wars
37559  Solo: A Star Wars Story | Tour The Millennium ...     Star Wars
37731  Solo: A Star Wars Story | Tour The Millennium ...     Star Wars

[141 rows x 2 columns]
Toplam kayıt sayısı:  141


In [7]:
# Beğenisi 2.000.000 ve üzerinde aynı zamanda çift olan kayıtların title ve likes bilgilerini listeler.
youtube.query("likes>=2000000 & likes % 2==0")[["title","likes"]]

,title,likes
1103,Ed Sheeran - Perfect (Official Music Video),2056268
1526,Ed Sheeran - Perfect (Official Music Video),2205826
1733,Ed Sheeran - Perfect (Official Music Video),2273358
1967,Ed Sheeran - Perfect (Official Music Video),2341126
2177,Ed Sheeran - Perfect (Official Music Video),2404512
...,...,...
37593,Maroon 5 - Girls Like You ft. Cardi B,2280516
37705,Childish Gambino - This Is America (Official V...,5286116
38056,Childish Gambino - This Is America (Official V...,5366150
38235,Childish Gambino - This Is America (Official V...,5426274


In [39]:
# comments_disabled alanında değeri True olanların title ve comments_disabled alanlarını listeler (son 10 kaydı listeler)
result=youtube[youtube["comments_disabled"]==1][["title","comments_disabled"]].tail(10)
print(result)
print(youtube[youtube["comments_disabled"]==1][["comments_disabled"]].count()) # Toplam kaç kayıt olduğunu yazar.

                                    title  comments_disabled
37858       This Is America: Women's Edit               True
37869  2018 FIFA World Cup | Forget | ITV               True
38035       This Is America: Women's Edit               True
38046  2018 FIFA World Cup | Forget | ITV               True
38214       This Is America: Women's Edit               True
38224  2018 FIFA World Cup | Forget | ITV               True
38382       This Is America: Women's Edit               True
38555       This Is America: Women's Edit               True
38730       This Is America: Women's Edit               True
38900       This Is America: Women's Edit               True
comments_disabled    683
dtype: int64


Yukarıdaki sonuca göre yorum yapma özelliği devre dışı bırakılmış toplam 683 kayıt olduğunu gözlemledik.

In [40]:
# ratings_disabled alanında değeri True olanların title ve comments_disabled alanlarını listeler (ilk 10 kaydı listeler)
result=youtube[youtube["ratings_disabled"]==1][["title","ratings_disabled"]].head(10)
print(result)
print(youtube[youtube["ratings_disabled"]==1][["ratings_disabled"]].count()) # Toplam kaç kayıt olduğunu yazar.

                               title  ratings_disabled
37333  This Is America: Women's Edit              True
37505  This Is America: Women's Edit              True
37683  This Is America: Women's Edit              True
37858  This Is America: Women's Edit              True
38035  This Is America: Women's Edit              True
38214  This Is America: Women's Edit              True
38382  This Is America: Women's Edit              True
38555  This Is America: Women's Edit              True
38730  This Is America: Women's Edit              True
38900  This Is America: Women's Edit              True
ratings_disabled    272
dtype: int64


Yukarıdaki sonuca göre değerlendirme özelliği devre dışı bırakılmış toplam 272 kayıt olduğunu gözlemledik.

In [42]:
# video_error_or_removed alanında değeri True olanları listeler (ilk 10 kaydı listeler)
result=youtube[youtube["video_error_or_removed"]==1]["video_error_or_removed"].head(10)
print(result)
print(youtube[youtube["video_error_or_removed"]==1][["video_error_or_removed"]].count()) # Toplam kaç kayıt olduğunu yazar.

31995    True
32169    True
32173    True
32337    True
32341    True
32506    True
32509    True
32676    True
32681    True
32844    True
Name: video_error_or_removed, dtype: bool
video_error_or_removed    69
dtype: int64


Yukarıdaki sonuca göre veri setindeki videolarda hata olan veya kaldırılmış olan toplam 69 kayıt olduğunu gözlemledik.

### 7. isnull() ve dropna() kullanımı

In [64]:
# Dataset'de yer alan toplam null değerlerin sayısını verir.
result=youtube.isnull().sum().sum()
print("Veri setinde yer alan toplam null değerlerin sayısı: ",result)

Veri setinde yer alan toplam null değerlerin sayısı:  612


In [62]:
# Sütuna göre null değerleri silecektir. Null değer bir tane olsa bile komple o sütunu silecektir.
result=youtube.dropna(axis=1)
print(result.columns)

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'tag_count'],
      dtype='object')


Yukarıda dropna işlemi yapıldıktan sonra datasetin kolonlarını tekrar listeleyince description alanı null değerler içerdiği için description alanının olmadığını görüyoruz.

### 8. sort_values() ve value_counts() kullanımı

In [5]:
# En fazla görüntülenen ilk 10 videonun kaydını gösterir
youtube.sort_values("views",ascending=False).head(10)[["title","views"]]

,title,views
28412,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,424538912
28212,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,413586699
28008,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,402650804
27811,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,392036878
27615,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,382401497
27424,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,372399338
27241,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,362111555
27052,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,349987176
26861,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,339629489
34460,"Te Bote Remix - Casper, Nio García, Darell, Ni...",337621571


In [6]:
# Her kategoride kaç adet video olduğunu gösterir.
result=youtube["category_id"].value_counts()
print(result)

10    13754
24     9124
22     2926
1      2577
26     1928
17     1907
23     1828
20     1788
25     1225
15      534
28      518
27      457
2       144
19       96
29       90
43       20
Name: category_id, dtype: int64


### 9. Pandas String İşlemleri

In [15]:
# tags içerisinde Water kelimesi içeren kayıtlarından ilk 10 tanesini listeler.
youtube[youtube["tags"].str.contains("Water")][["title","tags"]].head(10)

,title,tags
2,Eminem - Walk On Water (Audio) ft. Beyoncé,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In..."
48,Thirty Seconds To Mars - Walk On Water (Offici...,"Thirty|""Seconds""|""To""|""Mars""|""Walk""|""On""|""Wate..."
219,Eminem - Walk On Water (Audio) ft. Beyoncé,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In..."
268,Thirty Seconds To Mars - Walk On Water (Offici...,"Thirty|""Seconds""|""To""|""Mars""|""Walk""|""On""|""Wate..."
427,Eminem - Walk On Water (Audio) ft. Beyoncé,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In..."
477,Thirty Seconds To Mars - Walk On Water (Offici...,"Thirty|""Seconds""|""To""|""Mars""|""Walk""|""On""|""Wate..."
666,Eminem - Walk On Water (Audio) ft. Beyoncé,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In..."
720,Thirty Seconds To Mars - Walk On Water (Offici...,"Thirty|""Seconds""|""To""|""Mars""|""Walk""|""On""|""Wate..."
876,Eminem - Walk On Water (Audio) ft. Beyoncé,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In..."
924,Thirty Seconds To Mars - Walk On Water (Offici...,"Thirty|""Seconds""|""To""|""Mars""|""Walk""|""On""|""Wate..."


In [32]:
# title alanındaki tüm verilerin hepsini büyük harfe çevirip ilk 10 kaydı listeledik.
youtube["title"].str.upper().head(10)

0        JOHN LEWIS CHRISTMAS AD 2017 - #MOZTHEMONSTER
1            TAYLOR SWIFT: …READY FOR IT? (LIVE) - SNL
2           EMINEM - WALK ON WATER (AUDIO) FT. BEYONCÉ
3    GOALS FROM SALFORD CITY VS CLASS OF 92 AND FRI...
4    DASHCAM CAPTURES TRUCK'S NEAR MISS WITH CHILD ...
5                         HOW MY RELATIONSHIP STARTED!
6          ED SHEERAN - PERFECT (OFFICIAL MUSIC VIDEO)
7                        CHRISTMAS HAS GONE TO MY HEAD
8     ELBOW - GOLDEN SLUMBERS (JOHN LEWIS ADVERT 2017)
9    JENNIFER LOPEZ - AMOR, AMOR, AMOR (OFFICIAL VI...
Name: title, dtype: object

In [2]:
# tags alanında [none] içeren değerleri null ile değiştirelim.
youtube[youtube["tags"]=="[none]"].replace("[none]"," ")[["title","tags"]]

,title,tags
4,Dashcam captures truck's near miss with child ...,
65,Waking Up with Sam Harris #103 - American Fant...,
86,The reputation Secret Sessions,
91,Bone on Labour HQ,
155,TYSON FURY TO ANTHONY JOSHUA - YOU AINT SEEN *...,
...,...,...
38750,BABYMETAL - Distortion (OFFICIAL),
38814,Bumblebee (2018) - Official Teaser Trailer - P...,
38825,Choco Mountain: The History of Mario Kart 64's...,
38884,Charmed (The CW) Trailer HD - 2018 Reboot,
